In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder

In [2]:
train_set = pd.read_csv('Datasets/train.csv')

train_set.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region,monthly_rent
0,2021-09,jurong east,257,Jurong East Street 24,3 room,new generation,67.0,yes,1983,1.344518,103.738630,0.0,yuhua east,jurong east,west region,1600
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,yes,1978,1.330186,103.938717,0.0,bedok north,bedok,east region,2250
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,yes,1971,1.332242,103.845643,0.0,toa payoh central,toa payoh,central region,1900
3,2021-08,pasir ris,250,Pasir Ris Street 21,executive,apartment,149.0,yes,1993,1.370239,103.962894,0.0,pasir ris drive,pasir ris,east region,2850
4,2022-11,kallang/whampoa,34,Whampoa West,3-room,improved,68.0,yes,1972,1.320502,103.863341,0.0,bendemeer,kallang,central region,2100


### data cleaning

In [3]:
# get copy

train_set_cleaned = train_set.copy()

In [4]:
# add two new attributes, split rent_approval_date into month, year

train_set_cleaned[['rent_approval_year','rent_approval_month']] = train_set_cleaned['rent_approval_date'].str.split('-',expand=True)

# train_set_cleaned['rent_approval_month'] = pd.to_datetime(train_set_cleaned['rent_approval_month'], format='%m').dt.month
# train_set_cleaned['rent_approval_year'] = pd.to_datetime(train_set_cleaned['rent_approval_year'], format='%y').dt.year

In [5]:
# street name to lower case

train_set_cleaned['street_name'] = train_set_cleaned['street_name'].apply(str.lower)

In [6]:
# replace blank space with hyphen in flat_type (e.g. '2 room' to '2-room')

train_set_cleaned['flat_type'] = train_set_cleaned['flat_type'].apply(lambda x: x.replace(' ', '-'))

In [7]:
def date_filter_condition(x):
    
    if x < 1970:
        return 'before 70s'
    elif x >= 1970 and x < 1980:
        return '70s'
    elif x >= 1980 and x < 1990:
        return '80s'
    elif x >= 1990 and x < 2000:
        return '90s'
    elif x >= 2000 and x < 2010:
        return '00s'
    elif x >= 2010 and x < 2020:
        return '10s'
    else:
        return 'others'

# categorize lease commence date by decades    
train_set_cleaned['lease_date_cat'] = train_set_cleaned['lease_commence_date'].apply(date_filter_condition)

In [8]:
train_set_cleaned.head(10)

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region,monthly_rent,rent_approval_year,rent_approval_month,lease_date_cat
0,2021-09,jurong east,257,jurong east street 24,3-room,new generation,67.0,yes,1983,1.344518,103.738630,0.0,yuhua east,jurong east,west region,1600,2021,09,80s
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,yes,1978,1.330186,103.938717,0.0,bedok north,bedok,east region,2250,2022,05,70s
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,yes,1971,1.332242,103.845643,0.0,toa payoh central,toa payoh,central region,1900,2022,10,70s
3,2021-08,pasir ris,250,pasir ris street 21,executive,apartment,149.0,yes,1993,1.370239,103.962894,0.0,pasir ris drive,pasir ris,east region,2850,2021,08,90s
4,2022-11,kallang/whampoa,34,whampoa west,3-room,improved,68.0,yes,1972,1.320502,103.863341,0.0,bendemeer,kallang,central region,2100,2022,11,70s
5,2023-04,bukit panjang,654,senja road,executive,premium apartment,130.0,yes,2001,1.387847,103.764249,0.0,saujana,bukit panjang,west region,2300,2023,04,00s
6,2021-01,sengkang,407b,fernvale road,5-room,premium apartment,110.0,yes,2005,1.388997,103.875148,0.0,fernvale,sengkang,north-east region,2100,2021,01,00s
7,2022-06,ang mo kio,223,ang mo kio avenue 1,3-room,new generation,67.0,yes,1978,1.366048,103.838123,0.0,shangri-la,ang mo kio,north-east region,2300,2022,06,70s
8,2021-10,bishan,149,bishan street 11,4-room,simplified,84.0,yes,1987,1.344279,103.855556,0.0,bishan east,bishan,central region,2100,2021,10,80s
9,2021-04,punggol,133,edgedale plains,5-room,premium apartment,112.0,yes,2003,1.392832,103.910620,0.0,punggol field,punggol,north-east region,2100,2021,04,00s


In [9]:
train_set_cleaned['rent_approval_year'].value_counts()

rent_approval_year
2021    24909
2022    21399
2023    13692
Name: count, dtype: int64

### encoding

In [10]:
# create a new dataframe
encoded_train_set = train_set_cleaned[['monthly_rent']].copy()

#### spatial information

In [11]:
# encode regions with the mean of target variable for each region

region_encoder = TargetEncoder()
encoded_train_set['region_encoded'] = region_encoder.fit_transform(train_set_cleaned['region'], train_set_cleaned['monthly_rent'])

In [12]:
# encode planning areas with target variable information

planning_area_encoder = TargetEncoder()
encoded_train_set['planning_area_encoded'] = planning_area_encoder.fit_transform(train_set_cleaned['planning_area'], train_set_cleaned['monthly_rent'])

In [13]:
# encode subzones with target variable information

subzone_encoder = TargetEncoder()
encoded_train_set['subzone_encoded'] = subzone_encoder.fit_transform(train_set_cleaned['subzone'], train_set_cleaned['monthly_rent'])

In [14]:
# encode regions with target variable information

street_encoder = TargetEncoder()
encoded_train_set['street_encoded'] = street_encoder.fit_transform(train_set_cleaned['street_name'], train_set_cleaned['monthly_rent'])

In [15]:
# spatial hierarchical correlation

region_planning_corr = encoded_train_set['region_encoded'].corr(encoded_train_set['planning_area_encoded'])
planning_subzone_corr = encoded_train_set['planning_area_encoded'].corr(encoded_train_set['subzone_encoded'])
subzone_street_corr = encoded_train_set['subzone_encoded'].corr(encoded_train_set['street_encoded'])

print(f'The correlation between region and planning area is {region_planning_corr:3f}')
print(f'The correlation between planning area and subzone is {planning_subzone_corr:3f}')
print(f'The correlation between subzone and street is {subzone_street_corr:3f}')

The correlation between region and planning area is 0.579100
The correlation between planning area and subzone is 0.669665
The correlation between subzone and street is 0.828036


In [16]:
# spatial information correlation with monthly rent

region_rent_corr = encoded_train_set['region_encoded'].corr(encoded_train_set['monthly_rent'])
planning_rent_corr = encoded_train_set['planning_area_encoded'].corr(encoded_train_set['monthly_rent'])
subzone_rent_corr = encoded_train_set['subzone_encoded'].corr(encoded_train_set['monthly_rent'])
street_rent_corr = encoded_train_set['street_encoded'].corr(encoded_train_set['monthly_rent'])

print(f'Region and Monthly Rental correlation is {region_rent_corr:3f}')
print(f'Planning Area and Monthly Rental correlation is {planning_rent_corr:3f}')
print(f'Subzone and Monthly Rental correlation is {subzone_rent_corr:3f}')
print(f'Street and Monthly Rental correlation is {street_rent_corr:3f}')

Region and Monthly Rental correlation is 0.124776
Planning Area and Monthly Rental correlation is 0.215462
Subzone and Monthly Rental correlation is 0.320711
Street and Monthly Rental correlation is 0.374731


#### rent date & lease date

In [17]:
# rental approval date

rental_encoder = TargetEncoder()
encoded_train_set['rental_approval_date_encoded'] = rental_encoder.fit_transform(train_set_cleaned['rent_approval_date'].astype(str), train_set_cleaned['monthly_rent'])

rental_date_rent_corr = encoded_train_set['rental_approval_date_encoded'].corr(encoded_train_set['monthly_rent'])
print(f'Rental Approval Date and Monthly Rental correlation is {rental_date_rent_corr:3f}')

Rental Approval Date and Monthly Rental correlation is 0.546074


In [18]:
# rental approval year

rental_year_encoder = TargetEncoder()
encoded_train_set['rental_approval_year_encoded'] = rental_year_encoder.fit_transform(train_set_cleaned['rent_approval_year'].astype(str), train_set_cleaned['monthly_rent'])

rental_year_rent_corr = encoded_train_set['rental_approval_year_encoded'].corr(encoded_train_set['monthly_rent'])
print(f'Rental Approval Year and Monthly Rental correlation is {rental_year_rent_corr:3f}')

Rental Approval Year and Monthly Rental correlation is 0.504737


In [19]:
# rental approval month

rental_month_encoder = TargetEncoder()
encoded_train_set['rental_approval_month_encoded'] = rental_month_encoder.fit_transform(train_set_cleaned['rent_approval_month'].astype(str), train_set_cleaned['monthly_rent'])

rental_month_rent_corr = encoded_train_set['rental_approval_month_encoded'].corr(encoded_train_set['monthly_rent'])
print(f'Rental Approval Month and Monthly Rental correlation is {rental_month_rent_corr:3f}')

Rental Approval Month and Monthly Rental correlation is 0.106482


In [20]:
# lease commence year (in decade)

lease_encoder = TargetEncoder()
encoded_train_set['lease_commence_date_encoded'] = lease_encoder.fit_transform(train_set_cleaned['lease_date_cat'], train_set_cleaned['monthly_rent'])

lease_date_rent_corr = encoded_train_set['lease_commence_date_encoded'].corr(encoded_train_set['monthly_rent'])
print(f'Lease Commence Date and Monthly Rental correlation is {lease_date_rent_corr:3f}')

Lease Commence Date and Monthly Rental correlation is 0.223300


#### flat_type and flat_model

In [21]:
train_set_cleaned['flat_type'].value_counts()

flat_type
4-room       21889
3-room       18897
5-room       14759
executive     3528
2-room         927
Name: count, dtype: int64

In [22]:
# flat type

flat_type_encoder = TargetEncoder()
encoded_train_set['flat_type_encoded'] = flat_type_encoder.fit_transform(train_set_cleaned['flat_type'], train_set_cleaned['monthly_rent'])

flat_type_rent_corr = encoded_train_set['flat_type_encoded'].corr(encoded_train_set['monthly_rent'])
print(f'Flat Type and Monthly Rental correlation is {flat_type_rent_corr:3f}')

Flat Type and Monthly Rental correlation is 0.346146


In [23]:
# flat model

flat_model_encoder = TargetEncoder()
encoded_train_set['flat_model_encoded'] = flat_model_encoder.fit_transform(train_set_cleaned['flat_model'], train_set_cleaned['monthly_rent'])

flat_model_rent_corr = encoded_train_set['flat_model_encoded'].corr(encoded_train_set['monthly_rent'])
print(f'Flat Model and Monthly Rental correlation is {flat_model_rent_corr:3f}')

Flat Model and Monthly Rental correlation is 0.236876


#### floor area

In [24]:
# floor area sqm
encoded_train_set['floor_area_sqm'] = train_set_cleaned['floor_area_sqm'].copy()

### normalization

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
encoded_train_set

,monthly_rent,region_encoded,planning_area_encoded,subzone_encoded,street_encoded,rental_approval_date_encoded,rental_approval_year_encoded,rental_approval_month_encoded,lease_commence_date_encoded,flat_type_encoded,flat_model_encoded,floor_area_sqm
0,1600,2569.167537,2595.146199,2542.158516,2312.179832,2233.926780,2225.773817,2489.108495,2479.803864,2276.033233,2369.965462,67.0
1,2250,2570.667785,2438.227223,2360.371046,2404.212860,2517.128874,2651.014066,2618.130520,2421.705462,2692.359176,2369.965462,92.0
2,1900,2737.201353,2516.680515,2808.893871,2403.464419,2928.483245,2651.014066,2563.328013,2421.705462,2276.033233,2636.211052,67.0
3,2850,2570.667785,2686.857477,2610.338573,2757.834101,2249.901768,2225.773817,2470.895522,2700.899570,2892.857143,2878.725962,149.0
4,2100,2737.201353,2702.635659,2793.525180,2407.998266,2986.739659,2651.014066,2611.993243,2421.705462,2276.033233,2636.211052,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2200,2558.822710,2416.700057,2390.887097,2336.012658,2233.926780,2225.773817,2489.108495,2421.705462,2276.033233,2369.965462,67.0
59996,4100,2737.201353,2904.113924,2694.936709,2763.731680,3178.128128,3158.694858,2608.904470,2880.707364,2692.359176,2612.031305,83.0
59997,2250,2570.667785,2638.489123,2602.823315,2509.602223,2582.606383,2651.014066,2658.595055,2479.803864,2815.593875,2636.211052,122.0
59998,4700,2570.667785,2438.227223,2434.379786,2591.443246,3069.581639,3158.694858,2490.553580,2421.705462,2815.593875,2444.223986,123.0


In [27]:
# initialize the feature scaler
scaler = StandardScaler()

normalized_features = scaler.fit_transform(encoded_train_set.iloc[:,1:])

In [28]:
df_describe = pd.DataFrame(normalized_features)
df_describe.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04
mean,3.644847e-15,3.031353e-15,1.777896e-15,-1.202238e-15,-1.717737e-16,-6.679102e-17,-2.394529e-16,2.234657e-15,1.636854e-15,-1.326346e-17,9.616012e-17
std,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00,1.000008e+00
min,-1.566138e+00,-1.248074e+00,-1.997123e+00,-2.296490e+00,-1.136227e+00,-1.010297e+00,-1.568908e+00,-2.255897e+00,-2.991037e+00,-3.995729e+00,-2.511392e+00
25%,-3.531893e-01,-8.052347e-01,-6.691684e-01,-6.786320e-01,-8.720157e-01,-1.010297e+00,-6.184322e-01,-6.923440e-01,-1.270077e+00,-7.507854e-01,-8.919552e-01
50%,-2.372201e-01,3.131269e-02,-1.654897e-02,-1.316916e-01,-3.898313e-01,1.681796e-01,2.440221e-01,-6.923440e-01,4.123100e-01,1.323593e-01,-6.147460e-02
75%,-2.204018e-01,3.667341e-01,2.406733e-01,4.403614e-01,1.227626e+00,1.681796e-01,8.967736e-01,9.533255e-01,9.103054e-01,2.772886e-01,6.444339e-01
max,1.646499e+00,6.297960e+00,6.454196e+00,6.316627e+00,1.721150e+00,1.575125e+00,1.828760e+00,1.818984e+00,1.222529e+00,1.030540e+01,5.004457e+00


In [29]:
# normalize target variable
y_scaler = StandardScaler()

target_variable = y_scaler.fit_transform(encoded_train_set['monthly_rent'].to_numpy().reshape(-1, 1))
target_variable = target_variable.reshape(-1)

In [30]:
df_describe = pd.DataFrame(target_variable)
df_describe.describe()

,0
count,6.000000e+04
mean,-5.826450e-17
std,1.000008e+00
min,-3.203684e+00
25%,-6.858655e-01
50%,-2.662290e-01
75%,5.730439e-01
max,6.098257e+00


### training

In [31]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

#### lasso regression model

In [32]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Lasso Regression with cross-validation
lasso = LassoCV(cv=kfold, random_state=42)
lasso.fit(normalized_features, target_variable)

LassoCV(cv=KFold(n_splits=5, random_state=42, shuffle=True), random_state=42)

In [33]:
# evaluate the model

print("Optimal alpha:", lasso.alpha_)
print("Mean cross-validated score of the best estimator:", lasso.score(normalized_features, target_variable))

Optimal alpha: 0.0022045081649800934
Mean cross-validated score of the best estimator: 0.5026472684759563


In [34]:
# making predictions 
y_pred = lasso.predict(normalized_features)

# evaluating the model
mse = mean_squared_error(target_variable, y_pred)
print(f"Lasso Regression Mean Squared Error: {mse}")
print(f"Model Coefficients: \n{lasso.coef_}")

Lasso Regression Mean Squared Error: 0.4973527315240436
Model Coefficients: 
[ 1.07341824e-01  8.02923488e-04  3.84111326e-02  1.94728342e-01
  5.31511063e-01  1.09286102e-02 -2.66962319e-04 -0.00000000e+00
  2.56129575e-01  0.00000000e+00  5.47936350e-02]


#### graident boosting tree

In [35]:
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# implement gradient boosting tree with cross validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(gbr, normalized_features, target_variable, cv=cv, scoring='neg_mean_squared_error')

gbr.fit(normalized_features, target_variable)

GradientBoostingRegressor(random_state=42)

In [36]:
mean_mse = np.mean(-cross_val_scores)
print(f"Gradient Boosting Regressor Mean Squared Error: {mean_mse}")

Gradient Boosting Regressor Mean Squared Error: 0.46812061212081335


In [37]:
print("Mean cross-validated score of the best regressor:", gbr.score(normalized_features, target_variable))

Mean cross-validated score of the best regressor: 0.5393054798182728


### test set evaluation

In [38]:
test_set = pd.read_csv('Datasets/test.csv')

test_set.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region
0,2023-01,hougang,245,hougang street 22,5-room,improved,121.0,yes,1984,1.358411,103.891722,0.0,lorong ah soo,hougang,north-east region
1,2022-09,sembawang,316,sembawang vista,4-room,model a,100.0,yes,1999,1.446343,103.820817,0.0,sembawang central,sembawang,north region
2,2023-07,clementi,708,Clementi West Street 2,4-room,new generation,91.0,yes,1980,1.305719,103.762168,0.0,clementi west,clementi,west region
3,2021-08,jurong east,351,Jurong East Street 31,3 room,model a,74.0,yes,1986,1.344832,103.730778,0.0,yuhua west,jurong east,west region
4,2022-03,jurong east,305,jurong east street 32,5-room,improved,121.0,yes,1983,1.345437,103.735241,0.0,yuhua west,jurong east,west region


In [39]:
(test_set.isna().sum(axis=1) > 0).sum()

0

In [40]:
# data preparation

test_set[['rent_approval_year','rent_approval_month']] = test_set['rent_approval_date'].str.split('-',expand=True)

test_set['rent_approval_date'] = test_set['rent_approval_date'].astype('Period[M]')

# street name to lower case
test_set['street_name'] = test_set['street_name'].apply(str.lower)

# replace blank space with hyphen in flat_type (e.g. '2 room' to '2-room')
test_set['flat_type'] = test_set['flat_type'].apply(lambda x: x.replace(' ', '-'))

# categorize lease commence date by decades (remember to date_filter_condition function above first)
test_set['lease_date_cat'] = test_set['lease_commence_date'].apply(date_filter_condition)

In [41]:
# encoding test set

encoded_test_set = pd.DataFrame(index=test_set.index)

encoded_test_set['region_encoded'] = region_encoder.transform(test_set['region'])
encoded_test_set['planning_area_encoded'] = planning_area_encoder.transform(test_set['planning_area'])
encoded_test_set['subzone_encoded'] = subzone_encoder.transform(test_set['subzone'])
encoded_test_set['street_encoded'] = street_encoder.transform(test_set['street_name'])

encoded_test_set['rental_approval_date_encoded'] = rental_encoder.transform(test_set['rent_approval_date'])
encoded_test_set['rental_approval_year_encoded'] = rental_year_encoder.transform(test_set['rent_approval_year'])
encoded_test_set['rental_approval_month_encoded'] = rental_month_encoder.transform(test_set['rent_approval_month'])

encoded_test_set['lease_commence_date_encoded'] = lease_encoder.transform(test_set['lease_date_cat'])

encoded_test_set['flat_type_encoded'] = flat_type_encoder.transform(test_set['flat_type'])
encoded_test_set['flat_model_encoded'] = flat_model_encoder.transform(test_set['flat_model'])

encoded_test_set['floor_area_sqm'] = test_set['floor_area_sqm'].copy()



In [42]:
encoded_test_set.head(10)

,region_encoded,planning_area_encoded,subzone_encoded,street_encoded,rental_approval_date_encoded,rental_approval_year_encoded,rental_approval_month_encoded,lease_commence_date_encoded,flat_type_encoded,flat_model_encoded,floor_area_sqm
0,2558.822710,2503.252886,2427.604167,2348.993316,2590.328333,3158.694858,2490.553580,2479.803864,2815.593875,2636.211052,121.0
1,2450.623806,2540.491010,2592.332790,2640.043972,2590.328333,2651.014066,2489.108495,2700.899570,2692.359176,2612.031305,100.0
2,2569.167537,2646.808979,2395.588235,2532.195122,2590.328333,3158.694858,2729.542350,2479.803864,2692.359176,2369.965462,91.0
3,2569.167537,2595.146199,2400.150150,2780.896042,2590.328333,2225.773817,2470.895522,2479.803864,2276.033233,2612.031305,74.0
4,2569.167537,2595.146199,2400.150150,2268.564360,2590.328333,2651.014066,2543.250298,2479.803864,2815.593875,2636.211052,121.0
5,2569.167537,2646.808979,2395.588235,2229.881779,2590.328333,2651.014066,2490.553580,2479.803864,2276.033233,2369.965462,67.0
6,2558.822710,2665.555556,2654.294479,2590.503432,2590.328333,2225.773817,2618.130520,2742.515391,2815.593875,2709.678998,110.0
7,2570.667785,2638.489123,2602.085865,3081.579221,2590.328333,2651.014066,2470.895522,2742.515391,2815.593875,3150.404313,108.0
8,2569.167537,2640.890551,2873.930481,2820.173278,2590.328333,2225.773817,2680.680614,2742.515391,2892.857143,2709.678998,133.0
9,2737.201353,2585.947712,2585.947712,2480.479822,2590.328333,3158.694858,2729.542350,2421.705462,2815.593875,2444.223986,120.0


In [43]:
# normalization

normalized_test_features = scaler.transform(encoded_test_set)

In [44]:
# lasso regressor prediction
lasso_y_pred = lasso.predict(normalized_test_features).reshape(-1, 1)
lasso_y_pred = y_scaler.inverse_transform(lasso_y_pred)

In [45]:
lasso_y_pred[:20]

array([[2638.07941832],
       [2584.61883667],
       [2601.29352993],
       [2379.70944645],
       [2589.78253261],
       [2081.74073879],
       [2756.97425016],
       [3030.36795825],
       [3009.71426825],
       [2879.12616047],
       [2494.58355694],
       [2329.07311285],
       [2537.64397099],
       [2757.65927939],
       [2645.87395555],
       [2684.60741198],
       [2237.91952287],
       [2989.70775342],
       [2765.80046726],
       [2648.36433851]])

In [46]:
# XGBoost prediction
gbr_y_pred = gbr.predict(normalized_test_features).reshape(-1, 1)
gbr_y_pred = y_scaler.inverse_transform(gbr_y_pred)

In [47]:
gbr_y_pred[:30]

array([[2623.72026836],
       [2591.650405  ],
       [2619.88358809],
       [2299.96589769],
       [2618.31453875],
       [2240.84711411],
       [2632.62192082],
       [3024.28558682],
       [2883.93607308],
       [2947.91970038],
       [2433.53263796],
       [2244.79164793],
       [2736.96339214],
       [2660.4886048 ],
       [2606.38332563],
       [2575.21268777],
       [2228.43972603],
       [3075.60306771],
       [2737.43612658],
       [2632.43488934],
       [2185.68289684],
       [2417.72984217],
       [2674.25231862],
       [2427.79251577],
       [2448.82243865],
       [2315.37280209],
       [2810.64536438],
       [2624.33752873],
       [2449.50436559],
       [2675.58103808]])

### XGBoost Regression

In [50]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

In [51]:
X, y = normalized_features, target_variable

In [52]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [53]:
# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

#Printing basic information about the DMatrix
print("Number of training samples in DMatrix:", dtrain_reg.num_row())
print("Number of features in Dmatrix:", dtrain_reg.num_col())

Number of training samples in DMatrix: 45000
Number of features in Dmatrix: 11


In [54]:
# Define hyperparameters
params = {
    "objective": "reg:squarederror",
}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

In [55]:
feature_score = model.get_score(importance_type="gain")
print(feature_score)

{'f0': 6.989086151123047, 'f1': 5.954370975494385, 'f2': 3.0978076457977295, 'f3': 5.976417541503906, 'f4': 21.886674880981445, 'f5': 6.861771106719971, 'f6': 1.217803955078125, 'f7': 5.4947638511657715, 'f8': 81.03535461425781, 'f9': 3.1352717876434326, 'f10': 3.252474069595337}


In [56]:
preds = model.predict(dtest_reg)

In [57]:
# compute and print accuracy score

rmse = mean_squared_error(y_test, preds, squared=False)
r2 = r2_score(y_test, preds)

print(f"RMSE of the base model: {rmse:.3f}")
print(f"R2 score of the base model: {r2:.3f}")

RMSE of the base model: 0.686
R2 score of the base model: 0.528


#### Early stopping
We can test our model at each step and see if adding a new tree/round improves performance. To do so, we define a test dataset and a metric that is used to assess performance at each round. If performance haven’t improved for N rounds (N is defined by the variable early_stopping_round), we stop the training and keep the best number of boosting rounds.

In [58]:
params = {
    "objective": "reg:squarederror",
}
n = 100

# evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]
evals = [(dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=2, # Every ten rounds
   early_stopping_rounds=10
)

[0]	validation-rmse:0.86118
[2]	validation-rmse:0.73987
[4]	validation-rmse:0.70222
[6]	validation-rmse:0.68987
[8]	validation-rmse:0.68512
[10]	validation-rmse:0.68233
[12]	validation-rmse:0.68113
[14]	validation-rmse:0.68025
[16]	validation-rmse:0.67937
[18]	validation-rmse:0.67916
[20]	validation-rmse:0.67904
[22]	validation-rmse:0.67893
[24]	validation-rmse:0.67852
[26]	validation-rmse:0.67863
[28]	validation-rmse:0.67855
[30]	validation-rmse:0.67877
[32]	validation-rmse:0.67887


#### XGBoost Cross Validation

In [59]:
params = {
    "objective": "reg:squarederror",
}
n = 100

results = xgb.cv(
   params, 
   dtrain_reg,
   num_boost_round=n,
   seed=42,
   nfold=5,
   metrics={'rmse'},
   early_stopping_rounds=20
)

In [60]:
best_rmse = results['test-rmse-mean'].min()
print(best_rmse)

0.6844837021090517


In [61]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,0.862514,0.000357,0.864458,0.002161
1,0.782984,0.000722,0.787350,0.000660
2,0.737122,0.001128,0.743983,0.001622
3,0.711005,0.001167,0.720058,0.002688
4,0.695496,0.001333,0.706681,0.003354


#### Optimize XGBoost Model using GridSearch
GridSearch is a method used for hyperparameter tuning where one specifies a subset of possible values for each hyperparameter of interest. The method then exhaustively tries out all possible combinations of these hyperparameters to find the combination that produces the best model performance, according to some metric.

##### max_depth & min_child_weight

In [62]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]
num_boost_round = 100

In [63]:
# Define initial best params and MAE
min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain_reg,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))


CV with max_depth=9, min_child_weight=5
	RMSE 0.6927293037534273 for 8 rounds
CV with max_depth=9, min_child_weight=6
	RMSE 0.6916230761624717 for 9 rounds
CV with max_depth=9, min_child_weight=7
	RMSE 0.6910707226028732 for 9 rounds
CV with max_depth=10, min_child_weight=5
	RMSE 0.6963424400096958 for 7 rounds
CV with max_depth=10, min_child_weight=6
	RMSE 0.6948687976994675 for 7 rounds
CV with max_depth=10, min_child_weight=7
	RMSE 0.6946273839050069 for 7 rounds
CV with max_depth=11, min_child_weight=5
	RMSE 0.7010052418879651 for 7 rounds
CV with max_depth=11, min_child_weight=6
	RMSE 0.6997439778497698 for 7 rounds
CV with max_depth=11, min_child_weight=7
	RMSE 0.6992920837608504 for 7 rounds
Best params: 9, 7, RMSE: 0.6910707226028732


In [64]:
params['max_depth'] = 9
params['min_child_weight'] = 7

##### gamma

In [65]:
min_rmse = float("Inf")
best_params = None
for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]:
    print("CV with gamma={}".format(gamma))
    # We update our parameters
    params['gamma'] = gamma
    # Run and time CV
    cv_results = xgb.cv(
        params,
        dtrain_reg,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics=['rmse'],
        early_stopping_rounds=10
    )
    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds\n".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = gamma
print("Best params: {}, rmse: {}".format(best_params, min_rmse))

CV with gamma=0.1
	RMSE 0.6908500081705464 for 8 rounds

CV with gamma=0.2
	RMSE 0.6918208686994228 for 9 rounds

CV with gamma=0.3
	RMSE 0.6914086801394561 for 9 rounds

CV with gamma=0.4
	RMSE 0.6907230320070361 for 8 rounds

CV with gamma=0.5
	RMSE 0.6907623126965649 for 8 rounds

CV with gamma=0.6
	RMSE 0.6908753673131326 for 8 rounds

Best params: 0.4, rmse: 0.6907230320070361


In [66]:
params['gamma'] = 0.4

##### subsample & colsample

In [67]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [68]:
min_rmse = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain_reg,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample,colsample)
print("Best params: {}, {}, rmse: {}".format(best_params[0], best_params[1], min_rmse))

CV with subsample=1.0, colsample=1.0
	RMSE 0.6907230320070361 for 8 rounds
CV with subsample=1.0, colsample=0.9
	RMSE 0.6907092141852995 for 9 rounds
CV with subsample=1.0, colsample=0.8
	RMSE 0.6902243087619436 for 9 rounds
CV with subsample=1.0, colsample=0.7
	RMSE 0.6890205327365949 for 11 rounds
CV with subsample=0.9, colsample=1.0
	RMSE 0.6915455246339459 for 10 rounds
CV with subsample=0.9, colsample=0.9
	RMSE 0.691085922808293 for 9 rounds
CV with subsample=0.9, colsample=0.8
	RMSE 0.6899423317047082 for 9 rounds
CV with subsample=0.9, colsample=0.7
	RMSE 0.6900960576520607 for 10 rounds
CV with subsample=0.8, colsample=1.0
	RMSE 0.6924958960204927 for 8 rounds
CV with subsample=0.8, colsample=0.9
	RMSE 0.6912269553003347 for 10 rounds
CV with subsample=0.8, colsample=0.8
	RMSE 0.6915278019112321 for 10 rounds
CV with subsample=0.8, colsample=0.7
	RMSE 0.6907504782446079 for 11 rounds
CV with subsample=0.7, colsample=1.0
	RMSE 0.6929313870514051 for 8 rounds
CV with subsample=0.

In [69]:
params['subsample'] = .8
params['colsample_bytree'] = .7

##### alpha & lumbda

In [70]:
gridsearch_params = {
    'reg_alpha': [0.05, 0.1, 1, 2, 3]
}

In [71]:
min_rmse = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for reg_alpha in gridsearch_params['reg_alpha']:
        print("CV with reg_alpha={}".format(
                                 reg_alpha))
        # We update our parameters
        params['reg_alpha'] = reg_alpha
        # Run CV
        cv_results = xgb.cv(
            params,
            dtrain_reg,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics={'rmse'},
            early_stopping_rounds=10
        )
        # Update best score
        mean_rmse = cv_results['test-rmse-mean'].min()
        boost_rounds = cv_results['test-rmse-mean'].argmin()
        print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
        if mean_rmse < min_rmse:
            min_rmse = mean_rmse
            best_params = reg_alpha
print("Best params: {}, rmse: {}".format(best_params, min_rmse))

CV with reg_alpha=0.05
	RMSE 0.6905332135275477 for 13 rounds
CV with reg_alpha=0.1
	RMSE 0.6909033438502294 for 12 rounds
CV with reg_alpha=1
	RMSE 0.6890524341088993 for 13 rounds
CV with reg_alpha=2
	RMSE 0.687924667074656 for 13 rounds
CV with reg_alpha=3
	RMSE 0.6868598971363127 for 15 rounds
Best params: 3, rmse: 0.6868598971363127


In [72]:
params['alpha'] = 3

In [73]:
%time

min_rmse = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time 
    cv_results = xgb.cv(
        params,
        dtrain_reg,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics=['rmse'],
        early_stopping_rounds=10
    )
    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds\n".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = eta
print("Best params: {}, rmse: {}".format(best_params, min_rmse))

CPU times: total: 0 ns
Wall time: 0 ns
CV with eta=0.3
CPU times: total: 0 ns
Wall time: 0 ns
	RMSE 0.6868598971363127 for 15 rounds

CV with eta=0.2
CPU times: total: 0 ns
Wall time: 0 ns
	RMSE 0.6858406253177308 for 24 rounds

CV with eta=0.1
CPU times: total: 0 ns
Wall time: 0 ns
	RMSE 0.6833021449627252 for 48 rounds

CV with eta=0.05
CPU times: total: 0 ns
Wall time: 0 ns
	RMSE 0.6830404850638035 for 99 rounds

CV with eta=0.01
CPU times: total: 0 ns
Wall time: 0 ns
	RMSE 0.7533193028419659 for 99 rounds

CV with eta=0.005
CPU times: total: 0 ns
Wall time: 0 ns
	RMSE 0.8348466494663433 for 99 rounds

Best params: 0.05, rmse: 0.6830404850638035


In [74]:
params['eta'] = .05

In [75]:
params

{'objective': 'reg:squarederror',
 'max_depth': 9,
 'min_child_weight': 7,
 'gamma': 0.4,
 'subsample': 0.8,
 'colsample_bytree': 0.7,
 'reg_alpha': 3,
 'alpha': 3,
 'eta': 0.05}

In [76]:
model = xgb.train(
    params,
    dtrain_reg,
    num_boost_round=num_boost_round,
    evals=[(dtest_reg, "Test")],
    early_stopping_rounds=10
)

[0]	Test-rmse:0.97382
[1]	Test-rmse:0.95221
[2]	Test-rmse:0.93068
[3]	Test-rmse:0.92129
[4]	Test-rmse:0.91268
[5]	Test-rmse:0.90490
[6]	Test-rmse:0.88681
[7]	Test-rmse:0.87225
[8]	Test-rmse:0.85668
[9]	Test-rmse:0.84367
[10]	Test-rmse:0.83037
[11]	Test-rmse:0.81848
[12]	Test-rmse:0.80691
[13]	Test-rmse:0.80230
[14]	Test-rmse:0.79285
[15]	Test-rmse:0.78348
[16]	Test-rmse:0.77463
[17]	Test-rmse:0.76640
[18]	Test-rmse:0.75910
[19]	Test-rmse:0.75339
[20]	Test-rmse:0.75052
[21]	Test-rmse:0.74785
[22]	Test-rmse:0.74255
[23]	Test-rmse:0.73708
[24]	Test-rmse:0.73258
[25]	Test-rmse:0.72840
[26]	Test-rmse:0.72422
[27]	Test-rmse:0.72064
[28]	Test-rmse:0.71781
[29]	Test-rmse:0.71430
[30]	Test-rmse:0.71128
[31]	Test-rmse:0.70837
[32]	Test-rmse:0.70571
[33]	Test-rmse:0.70358
[34]	Test-rmse:0.70255
[35]	Test-rmse:0.70070
[36]	Test-rmse:0.69877
[37]	Test-rmse:0.69723
[38]	Test-rmse:0.69553
[39]	Test-rmse:0.69403
[40]	Test-rmse:0.69277
[41]	Test-rmse:0.69157
[42]	Test-rmse:0.69038
[43]	Test-rmse:0.6897

In [77]:
print("Best RMSE: {:.3f} in {} rounds".format(model.best_score, model.best_iteration+1))

Best RMSE: 0.679 in 100 rounds


#### Saving the best model

In [78]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain_reg,
    num_boost_round=num_boost_round,
    evals=[(dtest_reg, "Test")]
)

[0]	Test-rmse:0.97382
[1]	Test-rmse:0.95221
[2]	Test-rmse:0.93068
[3]	Test-rmse:0.92129
[4]	Test-rmse:0.91268
[5]	Test-rmse:0.90490
[6]	Test-rmse:0.88681
[7]	Test-rmse:0.87225
[8]	Test-rmse:0.85668
[9]	Test-rmse:0.84367
[10]	Test-rmse:0.83037
[11]	Test-rmse:0.81848
[12]	Test-rmse:0.80691
[13]	Test-rmse:0.80230
[14]	Test-rmse:0.79285
[15]	Test-rmse:0.78348
[16]	Test-rmse:0.77463
[17]	Test-rmse:0.76640
[18]	Test-rmse:0.75910
[19]	Test-rmse:0.75339
[20]	Test-rmse:0.75052
[21]	Test-rmse:0.74785
[22]	Test-rmse:0.74255
[23]	Test-rmse:0.73708
[24]	Test-rmse:0.73258
[25]	Test-rmse:0.72840
[26]	Test-rmse:0.72422
[27]	Test-rmse:0.72064
[28]	Test-rmse:0.71781
[29]	Test-rmse:0.71430
[30]	Test-rmse:0.71128
[31]	Test-rmse:0.70837
[32]	Test-rmse:0.70571
[33]	Test-rmse:0.70358
[34]	Test-rmse:0.70255
[35]	Test-rmse:0.70070
[36]	Test-rmse:0.69877
[37]	Test-rmse:0.69723
[38]	Test-rmse:0.69553
[39]	Test-rmse:0.69403
[40]	Test-rmse:0.69277
[41]	Test-rmse:0.69157
[42]	Test-rmse:0.69038
[43]	Test-rmse:0.6897

In [79]:
preds = best_model.predict(dtest_reg)

In [80]:
# compute and print accuracy score

rmse = mean_squared_error(y_test, preds, squared=False)
r2 = r2_score(y_test, preds)
mape = mean_absolute_percentage_error(y_test, preds)

print(f"RMSE of the base model: {rmse:.3f}")
print(f"R2 score of the base model: {r2:.3f}")

RMSE of the base model: 0.679
R2 score of the base model: 0.538
